In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

Using TensorFlow backend.


In [2]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [3]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [4]:
from keras.callbacks import ModelCheckpoint,LearningRateScheduler
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(32, (3, 3),input_shape=(75, 75, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))
    return model
print('model model')


model model


In [7]:
from sklearn.model_selection import KFold
from keras.preprocessing.image import ImageDataGenerator

def lr_f(epoch):
    if epoch<10:
        return 0.001
    elif epoch<30:
        return 0.0005
    else:
        return 0.0001

def kfold_train(fold_cnt=3,rnd=233):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        val_x,val_y = train_x[test_index],y[test_index]
        datagen = ImageDataGenerator(
            width_shift_range=0.05,
            height_shift_range=0.05,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True
        )
        
        
        bat_size = 16
        steps_train = len(curr_y)//bat_size
        
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        lr_s = LearningRateScheduler(lr_f)
        model.fit_generator(datagen.flow(curr_x, curr_y, batch_size=bat_size),
                  validation_data=(val_x,val_y),
                  steps_per_epoch = steps_train,
                  epochs=100, 
                  verbose=2,
                  callbacks=[model_chk,lr_s]
                 )
        
        
        model = load_model(model_p)
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
        print('============================')
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

Epoch 1/100
Epoch 00001: val_loss improved from inf to 0.77881, saving model to best_m.h5
 - 2s - loss: 0.6578 - acc: 0.5574 - val_loss: 0.7788 - val_acc: 0.4514
Epoch 2/100
Epoch 00002: val_loss improved from 0.77881 to 0.46585, saving model to best_m.h5
 - 1s - loss: 0.5885 - acc: 0.6547 - val_loss: 0.4659 - val_acc: 0.7656
Epoch 3/100
Epoch 00003: val_loss improved from 0.46585 to 0.44119, saving model to best_m.h5
 - 1s - loss: 0.4905 - acc: 0.7878 - val_loss: 0.4412 - val_acc: 0.8080
Epoch 4/100
Epoch 00004: val_loss improved from 0.44119 to 0.32279, saving model to best_m.h5
 - 1s - loss: 0.4347 - acc: 0.8091 - val_loss: 0.3228 - val_acc: 0.8429
Epoch 5/100
Epoch 00005: val_loss did not improve
 - 1s - loss: 0.3784 - acc: 0.8372 - val_loss: 0.3555 - val_acc: 0.8204
Epoch 6/100
Epoch 00006: val_loss improved from 0.32279 to 0.29912, saving model to best_m.h5
 - 1s - loss: 0.3770 - acc: 0.8239 - val_loss: 0.2991 - val_acc: 0.8653
Epoch 7/100
Epoch 00007: val_loss did not improve
 -

Epoch 62/100
Epoch 00062: val_loss did not improve
 - 1s - loss: 0.1991 - acc: 0.9150 - val_loss: 0.2283 - val_acc: 0.9052
Epoch 63/100
Epoch 00063: val_loss did not improve
 - 1s - loss: 0.1864 - acc: 0.9200 - val_loss: 0.2089 - val_acc: 0.9177
Epoch 64/100
Epoch 00064: val_loss did not improve
 - 1s - loss: 0.2023 - acc: 0.9156 - val_loss: 0.2027 - val_acc: 0.9077
Epoch 65/100
Epoch 00065: val_loss did not improve
 - 1s - loss: 0.2002 - acc: 0.9167 - val_loss: 0.2050 - val_acc: 0.9027
Epoch 66/100
Epoch 00066: val_loss did not improve
 - 1s - loss: 0.1848 - acc: 0.9187 - val_loss: 0.2062 - val_acc: 0.9227
Epoch 67/100
Epoch 00067: val_loss did not improve
 - 1s - loss: 0.2037 - acc: 0.9192 - val_loss: 0.2119 - val_acc: 0.9127
Epoch 68/100
Epoch 00068: val_loss did not improve
 - 1s - loss: 0.2147 - acc: 0.9158 - val_loss: 0.2136 - val_acc: 0.9102
Epoch 69/100
Epoch 00069: val_loss did not improve
 - 1s - loss: 0.1917 - acc: 0.9200 - val_loss: 0.2071 - val_acc: 0.9102
Epoch 70/100
Epo

Epoch 25/100
Epoch 00025: val_loss did not improve
 - 1s - loss: 0.2149 - acc: 0.9125 - val_loss: 0.2688 - val_acc: 0.9177
Epoch 26/100
Epoch 00026: val_loss did not improve
 - 1s - loss: 0.2240 - acc: 0.9033 - val_loss: 0.2852 - val_acc: 0.8928
Epoch 27/100
Epoch 00027: val_loss did not improve
 - 1s - loss: 0.2189 - acc: 0.9108 - val_loss: 0.2536 - val_acc: 0.9077
Epoch 28/100
Epoch 00028: val_loss did not improve
 - 1s - loss: 0.2802 - acc: 0.8662 - val_loss: 0.2635 - val_acc: 0.8978
Epoch 29/100
Epoch 00029: val_loss did not improve
 - 1s - loss: 0.2032 - acc: 0.9200 - val_loss: 0.2499 - val_acc: 0.9102
Epoch 30/100
Epoch 00030: val_loss did not improve
 - 1s - loss: 0.2202 - acc: 0.9003 - val_loss: 0.3314 - val_acc: 0.8853
Epoch 31/100
Epoch 00031: val_loss did not improve
 - 1s - loss: 0.2022 - acc: 0.9150 - val_loss: 0.2583 - val_acc: 0.9052
Epoch 32/100
Epoch 00032: val_loss did not improve
 - 1s - loss: 0.2126 - acc: 0.9083 - val_loss: 0.2580 - val_acc: 0.9052
Epoch 33/100
Epo

Epoch 92/100
Epoch 00092: val_loss did not improve
 - 1s - loss: 0.1520 - acc: 0.9375 - val_loss: 0.2557 - val_acc: 0.9152
Epoch 93/100
Epoch 00093: val_loss did not improve
 - 1s - loss: 0.1797 - acc: 0.9217 - val_loss: 0.2628 - val_acc: 0.9152
Epoch 94/100
Epoch 00094: val_loss did not improve
 - 1s - loss: 0.1716 - acc: 0.9298 - val_loss: 0.2560 - val_acc: 0.9077
Epoch 95/100
Epoch 00095: val_loss did not improve
 - 1s - loss: 0.1638 - acc: 0.9325 - val_loss: 0.2673 - val_acc: 0.9077
Epoch 96/100
Epoch 00096: val_loss did not improve
 - 1s - loss: 0.1536 - acc: 0.9358 - val_loss: 0.2649 - val_acc: 0.9002
Epoch 97/100
Epoch 00097: val_loss did not improve
 - 1s - loss: 0.1441 - acc: 0.9475 - val_loss: 0.2795 - val_acc: 0.8978
Epoch 98/100
Epoch 00098: val_loss did not improve
 - 1s - loss: 0.1745 - acc: 0.9214 - val_loss: 0.2629 - val_acc: 0.9202
Epoch 99/100
Epoch 00099: val_loss did not improve
 - 1s - loss: 0.1519 - acc: 0.9341 - val_loss: 0.2679 - val_acc: 0.9127
Epoch 100/100
Ep

Epoch 55/100
Epoch 00055: val_loss did not improve
 - 1s - loss: 0.2114 - acc: 0.9108 - val_loss: 0.1965 - val_acc: 0.9252
Epoch 56/100
Epoch 00056: val_loss did not improve
 - 1s - loss: 0.1801 - acc: 0.9216 - val_loss: 0.1955 - val_acc: 0.9252
Epoch 57/100
Epoch 00057: val_loss improved from 0.19499 to 0.19431, saving model to best_m.h5
 - 1s - loss: 0.1954 - acc: 0.9150 - val_loss: 0.1943 - val_acc: 0.9252
Epoch 58/100
Epoch 00058: val_loss did not improve
 - 1s - loss: 0.1855 - acc: 0.9225 - val_loss: 0.2583 - val_acc: 0.9052
Epoch 59/100
Epoch 00059: val_loss did not improve
 - 1s - loss: 0.1759 - acc: 0.9192 - val_loss: 0.2002 - val_acc: 0.9252
Epoch 60/100
Epoch 00060: val_loss did not improve
 - 1s - loss: 0.1951 - acc: 0.9117 - val_loss: 0.2003 - val_acc: 0.9252
Epoch 61/100
Epoch 00061: val_loss did not improve
 - 1s - loss: 0.2010 - acc: 0.9108 - val_loss: 0.1950 - val_acc: 0.9302
Epoch 62/100
Epoch 00062: val_loss improved from 0.19431 to 0.18929, saving model to best_m.h5


Epoch 17/100
Epoch 00017: val_loss did not improve
 - 1s - loss: 0.2702 - acc: 0.8773 - val_loss: 0.3101 - val_acc: 0.8554
Epoch 18/100
Epoch 00018: val_loss did not improve
 - 1s - loss: 0.2654 - acc: 0.8933 - val_loss: 0.3418 - val_acc: 0.8554
Epoch 19/100
Epoch 00019: val_loss improved from 0.28746 to 0.27655, saving model to best_m.h5
 - 1s - loss: 0.2426 - acc: 0.9042 - val_loss: 0.2766 - val_acc: 0.8628
Epoch 20/100
Epoch 00020: val_loss did not improve
 - 1s - loss: 0.2599 - acc: 0.8956 - val_loss: 0.2952 - val_acc: 0.8529
Epoch 21/100
Epoch 00021: val_loss did not improve
 - 1s - loss: 0.2386 - acc: 0.9042 - val_loss: 0.2970 - val_acc: 0.8653
Epoch 22/100
Epoch 00022: val_loss did not improve
 - 1s - loss: 0.2589 - acc: 0.8848 - val_loss: 0.2809 - val_acc: 0.8628
Epoch 23/100
Epoch 00023: val_loss improved from 0.27655 to 0.27269, saving model to best_m.h5
 - 1s - loss: 0.2512 - acc: 0.8950 - val_loss: 0.2727 - val_acc: 0.8753
Epoch 24/100
Epoch 00024: val_loss did not improve


Epoch 79/100
Epoch 00079: val_loss did not improve
 - 1s - loss: 0.2065 - acc: 0.9133 - val_loss: 0.2433 - val_acc: 0.9027
Epoch 80/100
Epoch 00080: val_loss did not improve
 - 1s - loss: 0.1775 - acc: 0.9208 - val_loss: 0.2801 - val_acc: 0.8853
Epoch 81/100
Epoch 00081: val_loss did not improve
 - 1s - loss: 0.1809 - acc: 0.9225 - val_loss: 0.2500 - val_acc: 0.8903
Epoch 82/100
Epoch 00082: val_loss did not improve
 - 1s - loss: 0.1745 - acc: 0.9292 - val_loss: 0.2457 - val_acc: 0.9077
Epoch 83/100
Epoch 00083: val_loss did not improve
 - 1s - loss: 0.1695 - acc: 0.9283 - val_loss: 0.2685 - val_acc: 0.8953
Epoch 84/100
Epoch 00084: val_loss did not improve
 - 1s - loss: 0.1925 - acc: 0.9173 - val_loss: 0.2425 - val_acc: 0.8878
Epoch 85/100
Epoch 00085: val_loss did not improve
 - 1s - loss: 0.1924 - acc: 0.9267 - val_loss: 0.2458 - val_acc: 0.8878
Epoch 86/100
Epoch 00086: val_loss did not improve
 - 1s - loss: 0.1839 - acc: 0.9192 - val_loss: 0.2523 - val_acc: 0.8903
Epoch 87/100
Epo

In [8]:
import pickle
with open('../features/cnn_2_aug1_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_2_aug1_sub.csv', index=False)
# pre 2249
# new 2166

0.216631708786
         id  is_iceberg
0  5941774d    0.093119
1  4023181e    0.592304
2  b20200e4    0.461044
3  e7f018bb    0.997086
4  4371c8c3    0.075144
